# MILESTONE 2
## REZA AMINI

* **Importing Libraries**

In [205]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [206]:
def price_simulator_multiple_trajectories(
                    random_seed:int,
                    num_asset:int,
                    init_price:np.ndarray,
                    interest_rate:np.ndarray,
                    sigma:np.ndarray,
                    num_stages:int,
                    delta_t:float,
                    num_trajectories:int):
    
    """
    *** GENERATING 3D PRICE MATRIX (J * T * M) ***
    
    random_seed: seed for random generator
    num_asset: number of assets (J)
    init_price: initial prices of assets
    interest_rate: interest rate of each asset
    sigma: stock price volatility
    num_stages: number of permissible exercise stages (T)
    delta_t: time difference between two subsequent stages
    num_trajectories: number of trajectories for each asset (M)
    
    """

    assert len(init_price) == num_asset
    assert len(interest_rate) == num_asset
    assert len(sigma) == num_asset

    np.random.seed(random_seed)
    prices = np.zeros([num_asset, num_stages, num_trajectories])
    for i in range(num_asset):
        prices[i, 0, :]    = init_price[i]*np.ones([num_trajectories])
    
    for i in range(1, int(num_stages)):
        for j in range(num_asset):
            drift           = (interest_rate[j] - 0.5 * sigma[j]**2) * delta_t
            Z               = np.random.normal(0., 1., num_trajectories)
            diffusion       = Z * (np.sqrt(delta_t)) * sigma[j]
            prices[j, i, :]    = prices[j, (i - 1), :]*np.exp(drift + diffusion)
    return prices

In [207]:
def plot_prices_multiple_trajectories(
                    random_seed:int,
                    prices:np.ndarray,
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int):
    
    """
    *** PLOTTING 3D PRICE MATRIX (J * T * M) ***
    
    random_seed: seed for random generator
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    
    """
        
    random.seed(random_seed)
    
    fig, ax  = plt.subplots(figsize=(6, 4),dpi=100)
    cols = ["g", "b", "r", "y", "k", "c", "m"]
    asset_color = random.sample(cols, num_asset)
    for i in range(num_asset):
        for j in range(num_trajectories):
            ax.plot(prices[i, :, j], lw = 2, alpha = .5, 
                    label = "Asset " + str(i + 1), marker = '.', color = asset_color[i])

    min_p,max_p = float(np.min(prices)), float(np.max(prices))
    ax.set_xticks(np.arange(0, num_stages + 1, 4))
    ax.set_yticks(np.round(np.arange(min_p, max_p + 1, int((max_p - min_p)/10.))))
    ax.set_xlabel(r'stage ($t$)')
    ax.set_ylabel(r'price ($p_{t,j}$)')
    plt.legend()
    plt.show()
    return True

In [208]:
def knock_out_indicator_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray, 
                    barrier_price:float):
    
    """
    *** GENERATING KNOCk-OUT INDICATOR MATRIX (M * T) ***
    
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    barrier_price: the barrier knock-out price value
    
    """
    
    y = np.zeros([num_trajectories, num_stages], dtype = np.int8)
    
    for i in range(num_trajectories):
        for j in range(num_stages):
            if j == 0:
                if np.max(prices[:, 0, i]) >= barrier_price:
                    y[i, 0] = 1
            else:
                if np.max(prices[:, j, i]) >= barrier_price or y[i, (j - 1)] == 1:
                    y[i, j] = 1
                    
    return y

In [209]:
def payoff_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    knock_out:np.ndarray,
                    strike_price:float):
    
    """
    *** GENERATING PAYOFF MATRIX (M * T) ***
    
    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    knock_out: knock-out indicator matrix
    strike_price: the target strike price
    
    """
    
    g = np.zeros([num_trajectories, num_stages])
    
    for i in range(num_trajectories):
        for j in range(num_stages):
            g[i, j] = np.max([np.max(prices[:, j, i]) - strike_price, 0.])*float(1 - knock_out[i, j])
            
    return g

In [210]:
def feature_matrix_calculator(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    knock_out:np.ndarray,
                    payoff:np.ndarray,
                    method:str,
                    centroid:np.ndarray = np.array([0.]),
                    rho:float = 0.5):
    
    """
    *** GENERATING FEATURE MATRIX (M * J * T) ***

    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    knock_out: knock-out indicator matrix
    payoff: payoff matrix
    method: continuation function approximation method
    centroid: radial basis function centroid for kernel method
    rho: bandwidth length for kernel method
    
    """
    
    if method == "linear":
        
        F = np.zeros([num_trajectories, num_asset, num_stages]) 
        for i in range(num_trajectories):
            for j in range(num_asset):
                F[i, j, :] = prices[j, :, i]*(1 - knock_out[i, :])
    if method == "DFM":
        
        F = np.zeros([num_trajectories, num_asset + 2, num_stages])
        for i in range(num_trajectories):
            for j in range(num_asset + 2):
                if j == 0:
                    F[i, j, :] = np.ones([num_stages])*(1 - knock_out[i, :])
                elif j == 1:
                    F[i, j, :] = payoff[i, :]*(1 - knock_out[i, :])
                else:
                    F[i, j, :] = prices[j - 2, :, i]*(1 - knock_out[i, :])
    
    if method == "kernel":
        F = np.zeros([num_trajectories, len(centroid), num_stages])
        for i in range(num_trajectories):
            for j in range(len(centroid)):
                for k in range(num_stages):
                    F[i, j, k] = np.exp(-np.linalg.norm(prices[:, k, i] - centroid[j], 2)**2/2./rho)*(1 - knock_out[i, k])
                
    return F

In [211]:
def fit_CFA(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    feature_matrix:np.ndarray,
                    payoff:np.ndarray,
                    gamma:float):
    
    """
    *** LSM TO FIT CFA ***

    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    feature_matrix: feature matrix
    payoff: payoff matrix
    gamma: discount factor
    
    """
    
    B = feature_matrix.shape[1]
    w = np.zeros([B, num_stages])
    w[:, (num_stages - 1)] = 0.
    gamma = np.exp(-delta_t*np.mean(interest_rate))
    for i in range(num_stages - 2, -1, -1):
        ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
        w[:, i], _, _, _ = np.linalg.lstsq(feature_matrix[:, :, i], ct, rcond = None)
    
    return w

In [212]:
def fit_CFA_regularizer(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    feature_matrix:np.ndarray,
                    payoff:np.ndarray,
                    gamma:float,
                    reg_lambda:float):
    
    """
    *** LSM TO FIT CFA ***

    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    feature_matrix: feature matrix
    payoff: payoff matrix
    gamma: discount factor
    
    """
    
    B = feature_matrix.shape[1]
    w = np.zeros([B, num_stages])
    w[:, (num_stages - 1)] = 0.
    gamma = np.exp(-delta_t*np.mean(interest_rate))
    for i in range(num_stages - 2, -1, -1):
        ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
        w[:, i], _, _, _ = np.linalg.lstsq(feature_matrix[:, :, i].T.dot(feature_matrix[:, :, i]) + reg_lambda*np.identity(B), feature_matrix[:, :, i].T.dot(ct), rcond = None)
    
    return w

In [213]:
def policy_simulation(
                    num_asset:int,
                    num_stages:int,
                    num_trajectories:int,
                    prices:np.ndarray,
                    weight:np.ndarray,
                    payoff:np.ndarray,
                    knock_out:np.ndarray,
                    feature_matrix:np.ndarray,
                    gamma:float):
    
    """
    *** SIMULATE POLICY AND RETURN AVERAGE PAYOFF ***

    prices: prices matrix
    num_asset: number of assets (J)
    num_stages: number of permissible exercise stages (T)
    num_trajectories: number of trajectories for each asset (M)
    weight: weights fitted to CFA
    knock_out: knock-out indicator matrix
    payoff: payoff matrix
    feature_matrix: feature matrix
    gamma: discount factor
    
    """
    
    c = np.zeros([num_trajectories, num_stages])
    
    r = np.ones([num_trajectories])*np.NaN
    for i in range(num_trajectories):
        for j in range(num_stages - 1):
            
            #c[i, j] = gamma*np.max([payoff[i, j + 1], (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j])])
            
            if (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j]) < payoff[i, j]:
                r[i] = gamma**(j + 1)*payoff[i, j]
                
        if np.isnan(r[i]):
            r[i] = gamma**num_stages*payoff[i, (num_stages - 1)]
            
    return np.sum(r)/num_trajectories

# Step 1

### Linear CFA - PI = 90 / J = 4

In [118]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  35.09002520440188


### Linear CFA - PI = 100 / J = 4

In [119]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  44.13018881407987


### Linear CFA - PI = 110 / J = 4

In [120]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  51.46580842239208


### Linear CFA - PI = 90 / J = 8

In [121]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  46.48306992680176


### Linear CFA - PI = 100 / J = 8

In [122]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  53.347468338792254


### Linear CFA - PI = 110 / J = 8

In [123]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  57.24233472183766


### DFM CFA - PI = 90 / J = 4

In [124]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  35.39161997628803


### DFM CFA - PI = 100 / J = 4

In [125]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  44.16993016494159


### DFM CFA - PI = 110 / J = 4

In [126]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  51.15672377273047


### DFM CFA - PI = 90 / J = 8

In [127]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  46.19902392124965


### DFM CFA - PI = 100 / J = 8

In [128]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  53.28398484908969


### DFM CFA - PI = 110 / J = 8

In [129]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  57.020941356482545


### Kernel CFA - PI = 90 / J = 4

In [130]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  34.95628202884407


### Kernel CFA - PI = 100 / J = 4

In [131]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  44.064762488122255


### Kernel CFA - PI = 110 / J = 4

In [132]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  51.298954153473275


### Kernel CFA - PI = 90 / J = 8

In [133]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  46.322979360751


### Kernel CFA - PI = 100 / J = 8

In [134]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  53.5451040983077


### Kernel CFA - PI = 110 / J = 8

In [135]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  57.52290960185151


# Step 2

### Linear CFA - PI = 90 / J = 4 - With Regularization

In [136]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  35.09002520440188


### Linear CFA - PI = 100 / J = 4 - With Regularization

In [137]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  44.13018881407987


### Linear CFA - PI = 110 / J = 4 - With Regularization

In [138]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  51.46580842239208


### Linear CFA - PI = 90 / J = 8 - With Regularization

In [139]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  46.48306992680176


### Linear CFA - PI = 100 / J = 8 - With Regularization

In [140]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  53.347468338792254


### Linear CFA - PI = 110 / J = 8 - With Regularization

In [141]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  57.24233472183766


### DFM CFA - PI = 90 / J = 4 - With Regularization

In [142]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  35.40282511909979


### DFM CFA - PI = 100 / J = 4 - With Regularization

In [143]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  44.18160871988758


### DFM CFA - PI = 110 / J = 4 - With Regularization

In [144]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  51.17382500136599


### DFM CFA - PI = 90 / J = 8 - With Regularization

In [145]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  46.21530729225667


### DFM CFA - PI = 100 / J = 8 - With Regularization

In [146]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  53.24846540717455


### DFM CFA - PI = 110 / J = 8 - With Regularization

In [147]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)




yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value = ", average_payoff)

Average payoff value =  57.091148091461804


### Kernek CFA - PI = 90 / J = 4 - With Regularization

In [148]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  34.958913309102


### Kernek CFA - PI = 100 / J = 4 - With Regularization

In [149]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  44.07824105255819


### Kernek CFA - PI = 110 / J = 4 - With Regularization

In [150]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  51.306993423120566


### Kernek CFA - PI = 90 / J = 8 - With Regularization

In [151]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  46.3364679181042


### Kernek CFA - PI = 100 / J = 8 - With Regularization

In [152]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  53.5451040983077


### Kernek CFA - PI = 110 / J = 8 - With Regularization

In [153]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 10000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 1/np.sqrt(num_asset)

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("Average payoff value: ", average_payoff)

Average payoff value:  57.52290960185151


# Step 3

### Linear CFA - PI = 90 / J = 4 - Cross-validation

In [154]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 35.0900252044
lambda = 0.1250,	average payoff = 35.0900252044
lambda = 0.2500,	average payoff = 35.0900252044
lambda = 0.5000,	average payoff = 35.0900252044
lambda = 1.0000,	average payoff = 35.0900252044
lambda = 2.0000,	average payoff = 35.0900252044
lambda = 4.0000,	average payoff = 35.0900252044
lambda = 8.0000,	average payoff = 35.0900252044
lambda = 16.0000,	average payoff = 35.0900252044
With lambda =  0.0625  Average payoff value is =  35.09002520440188


### Linear CFA - PI = 100 / J = 4 - Cross-validation

In [155]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 44.1301888141
lambda = 0.1250,	average payoff = 44.1301888141
lambda = 0.2500,	average payoff = 44.1301888141
lambda = 0.5000,	average payoff = 44.1301888141
lambda = 1.0000,	average payoff = 44.1301888141
lambda = 2.0000,	average payoff = 44.1301888141
lambda = 4.0000,	average payoff = 44.1301888141
lambda = 8.0000,	average payoff = 44.1301888141
lambda = 16.0000,	average payoff = 44.1301888141
With lambda =  0.0625  Average payoff value is =  44.13018881407987


### Linear CFA - PI = 110 / J = 4 - Cross-validation

In [156]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 51.4658084224
lambda = 0.1250,	average payoff = 51.4658084224
lambda = 0.2500,	average payoff = 51.4658084224
lambda = 0.5000,	average payoff = 51.4658084224
lambda = 1.0000,	average payoff = 51.4658084224
lambda = 2.0000,	average payoff = 51.4658084224
lambda = 4.0000,	average payoff = 51.4658084224
lambda = 8.0000,	average payoff = 51.4658084224
lambda = 16.0000,	average payoff = 51.4658084224
With lambda =  0.0625  Average payoff value is =  51.46580842239208


### Linear CFA - PI = 90 / J = 8 - Cross-validation

In [157]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 46.4830699268
lambda = 0.1250,	average payoff = 46.4830699268
lambda = 0.2500,	average payoff = 46.4830699268
lambda = 0.5000,	average payoff = 46.4830699268
lambda = 1.0000,	average payoff = 46.4830699268
lambda = 2.0000,	average payoff = 46.4830699268
lambda = 4.0000,	average payoff = 46.4830699268
lambda = 8.0000,	average payoff = 46.4830699268
lambda = 16.0000,	average payoff = 46.4830699268
With lambda =  0.0625  Average payoff value is =  46.48306992680176


### Linear CFA - PI = 100 / J = 8 - Cross-validation

In [158]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 53.3474683388
lambda = 0.1250,	average payoff = 53.3474683388
lambda = 0.2500,	average payoff = 53.3474683388
lambda = 0.5000,	average payoff = 53.3474683388
lambda = 1.0000,	average payoff = 53.3474683388
lambda = 2.0000,	average payoff = 53.3474683388
lambda = 4.0000,	average payoff = 53.3474683388
lambda = 8.0000,	average payoff = 53.3474683388
lambda = 16.0000,	average payoff = 53.3474683388
With lambda =  0.0625  Average payoff value is =  53.347468338792254


### Linear CFA - PI = 110 / J = 8 - Cross-validation

In [159]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "linear")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "linear")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "linear")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 57.2423347218
lambda = 0.1250,	average payoff = 57.2423347218
lambda = 0.2500,	average payoff = 57.2423347218
lambda = 0.5000,	average payoff = 57.2423347218
lambda = 1.0000,	average payoff = 57.2423347218
lambda = 2.0000,	average payoff = 57.2423347218
lambda = 4.0000,	average payoff = 57.2423347218
lambda = 8.0000,	average payoff = 57.2423347218
lambda = 16.0000,	average payoff = 57.2423347218
With lambda =  0.0625  Average payoff value is =  57.24233472183766


### DFM CFA - PI = 90 / J = 4 - Cross-validation

In [160]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 35.4029679588
lambda = 0.1250,	average payoff = 35.4022176064
lambda = 0.2500,	average payoff = 35.4022176064
lambda = 0.5000,	average payoff = 35.4028251191
lambda = 1.0000,	average payoff = 35.4051553736
lambda = 2.0000,	average payoff = 35.4100559682
lambda = 4.0000,	average payoff = 35.4303762733
lambda = 8.0000,	average payoff = 35.4437948358
lambda = 16.0000,	average payoff = 35.4306492207
With lambda =  8.0  Average payoff value is =  35.443794835778476


### DFM CFA - PI = 100 / J = 4 - Cross-validation

In [161]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 44.1713552085
lambda = 0.1250,	average payoff = 44.1723500518
lambda = 0.2500,	average payoff = 44.1802700192
lambda = 0.5000,	average payoff = 44.1816087199
lambda = 1.0000,	average payoff = 44.2494003855
lambda = 2.0000,	average payoff = 44.1967437467
lambda = 4.0000,	average payoff = 44.1851553674
lambda = 8.0000,	average payoff = 44.1952163055
lambda = 16.0000,	average payoff = 44.2004322535
With lambda =  1.0  Average payoff value is =  44.249400385500415


### DFM CFA - PI = 110 / J = 4 - Cross-validation

In [162]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 51.1500557117
lambda = 0.1250,	average payoff = 51.1572469939
lambda = 0.2500,	average payoff = 51.1566897866
lambda = 0.5000,	average payoff = 51.1738250014
lambda = 1.0000,	average payoff = 51.2026069366
lambda = 2.0000,	average payoff = 51.2238671132
lambda = 4.0000,	average payoff = 51.2259171230
lambda = 8.0000,	average payoff = 51.2038138333
lambda = 16.0000,	average payoff = 51.1837526114
With lambda =  4.0  Average payoff value is =  51.22591712303728


### DFM CFA - PI = 90 / J = 8 - Cross-validation

In [163]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 46.1990239212
lambda = 0.1250,	average payoff = 46.2022263811
lambda = 0.2500,	average payoff = 46.1966786424
lambda = 0.5000,	average payoff = 46.1956325461
lambda = 1.0000,	average payoff = 46.2057250432
lambda = 2.0000,	average payoff = 46.2164781018
lambda = 4.0000,	average payoff = 46.2349134790
lambda = 8.0000,	average payoff = 46.2962765181
lambda = 16.0000,	average payoff = 46.3188511084
With lambda =  16.0  Average payoff value is =  46.31885110842348


### DFM CFA - PI = 100 / J = 8 - Cross-validation

In [164]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 53.2675368916
lambda = 0.1250,	average payoff = 53.2727957159
lambda = 0.2500,	average payoff = 53.2484654072
lambda = 0.5000,	average payoff = 53.2646077025
lambda = 1.0000,	average payoff = 53.2936770228
lambda = 2.0000,	average payoff = 53.2359142321
lambda = 4.0000,	average payoff = 53.1224855111
lambda = 8.0000,	average payoff = 53.0800005333
lambda = 16.0000,	average payoff = 53.0062213514
With lambda =  1.0  Average payoff value is =  53.2936770228171


### DFM CFA - PI = 110 / J = 8 - Cross-validation

In [165]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "DFM")

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "DFM")


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "DFM")


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 57.0358310635
lambda = 0.1250,	average payoff = 57.0337271273
lambda = 0.2500,	average payoff = 57.0604858871
lambda = 0.5000,	average payoff = 57.1096186188
lambda = 1.0000,	average payoff = 57.1191862351
lambda = 2.0000,	average payoff = 57.0458051861
lambda = 4.0000,	average payoff = 57.0843448689
lambda = 8.0000,	average payoff = 57.0325120158
lambda = 16.0000,	average payoff = 57.0055057572
With lambda =  1.0  Average payoff value is =  57.11918623506614


### Kernel CFA - PI = 90 / J = 4 - Cross-validation

In [166]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 34.9605634676
lambda = 0.1250,	average payoff = 34.9605634676
lambda = 0.2500,	average payoff = 34.9597544804
lambda = 0.5000,	average payoff = 34.9586583854
lambda = 1.0000,	average payoff = 34.9714406605
lambda = 2.0000,	average payoff = 34.9896567250
lambda = 4.0000,	average payoff = 35.0118413852
lambda = 8.0000,	average payoff = 35.0224459103
lambda = 16.0000,	average payoff = 35.0849900322
With lambda =  16.0  Average payoff value is =  35.08499003223851


### Kernel CFA - PI = 100 / J = 4 - Cross-validation

In [167]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 44.0610492914
lambda = 0.1250,	average payoff = 44.0610492914
lambda = 0.2500,	average payoff = 44.0710205551
lambda = 0.5000,	average payoff = 44.0774944695
lambda = 1.0000,	average payoff = 44.0921768730
lambda = 2.0000,	average payoff = 44.1016809622
lambda = 4.0000,	average payoff = 44.1155664030
lambda = 8.0000,	average payoff = 44.1856296718
lambda = 16.0000,	average payoff = 44.2883169579
With lambda =  16.0  Average payoff value is =  44.28831695790259


### Kernel CFA - PI = 110 / J = 4 - Cross-validation

In [168]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 51.2989541535
lambda = 0.1250,	average payoff = 51.2989541535
lambda = 0.2500,	average payoff = 51.3029601534
lambda = 0.5000,	average payoff = 51.3069934231
lambda = 1.0000,	average payoff = 51.2998446704
lambda = 2.0000,	average payoff = 51.3080647393
lambda = 4.0000,	average payoff = 51.3269081741
lambda = 8.0000,	average payoff = 51.4040670396
lambda = 16.0000,	average payoff = 51.3941348256
With lambda =  8.0  Average payoff value is =  51.40406703959763


### Kernel CFA - PI = 90 / J = 8 - Cross-validation

In [169]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 46.3229793608
lambda = 0.1250,	average payoff = 46.3309421221
lambda = 0.2500,	average payoff = 46.3309421221
lambda = 0.5000,	average payoff = 46.3409626506
lambda = 1.0000,	average payoff = 46.3405812644
lambda = 2.0000,	average payoff = 46.3415634117
lambda = 4.0000,	average payoff = 46.3856698762
lambda = 8.0000,	average payoff = 46.4141578733
lambda = 16.0000,	average payoff = 46.4549628843
With lambda =  16.0  Average payoff value is =  46.454962884284505


### Kernel CFA - PI = 100 / J = 8 - Cross-validation

In [170]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)


    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 53.5451040983
lambda = 0.1250,	average payoff = 53.5451040983
lambda = 0.2500,	average payoff = 53.5451040983
lambda = 0.5000,	average payoff = 53.5572756169
lambda = 1.0000,	average payoff = 53.5572756169
lambda = 2.0000,	average payoff = 53.5580566626
lambda = 4.0000,	average payoff = 53.5605307708
lambda = 8.0000,	average payoff = 53.5472380566
lambda = 16.0000,	average payoff = 53.5624344688
With lambda =  16.0  Average payoff value is =  53.56243446880071


### Kernel CFA - PI = 110 / J = 8 - Cross-validation

In [171]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 1000000000. # bandwidth for kernel method
centroid         = [np.percentile(prices, 50)] # centroid for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 57.5229096019
lambda = 0.1250,	average payoff = 57.5229096019
lambda = 0.2500,	average payoff = 57.5229096019
lambda = 0.5000,	average payoff = 57.5229096019
lambda = 1.0000,	average payoff = 57.5202949974
lambda = 2.0000,	average payoff = 57.5339573710
lambda = 4.0000,	average payoff = 57.5234280926
lambda = 8.0000,	average payoff = 57.5063046448
lambda = 16.0000,	average payoff = 57.4884899914
With lambda =  2.0  Average payoff value is =  57.53395737103959


# Step 4

### Kernel CFA - PI = 90 / J = 4 / B = 19 / Optimal Bandwidth

In [173]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 34.0646311276
rho = 0.1250,	average payoff = 34.0415552779
rho = 0.2500,	average payoff = 34.0036563546
rho = 0.5000,	average payoff = 33.9607516512
rho = 1.0000,	average payoff = 33.9510766711
rho = 2.0000,	average payoff = 33.9503650530
rho = 4.0000,	average payoff = 33.9506606053
rho = 8.0000,	average payoff = 33.9519805491
rho = 16.0000,	average payoff = 33.9552414009
With rho =  0.0625  Average payoff value is =  34.04832327408291


### Kernel CFA - PI = 90 / J = 4 / B = 9 / Optimal Bandwidth

In [174]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 34.0744194500
rho = 0.1250,	average payoff = 34.0660867420
rho = 0.2500,	average payoff = 34.0258458028
rho = 0.5000,	average payoff = 33.9563237998
rho = 1.0000,	average payoff = 33.9422112012
rho = 2.0000,	average payoff = 33.9348592776
rho = 4.0000,	average payoff = 33.9294904337
rho = 8.0000,	average payoff = 33.9308460517
rho = 16.0000,	average payoff = 33.9453573118
With rho =  0.0625  Average payoff value is =  34.05175378380267


### Kernel CFA - PI = 90 / J = 4 / B = 3 / Optimal Bandwidth

In [175]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 4, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 34.0784065488
rho = 0.1250,	average payoff = 34.0752749300
rho = 0.2500,	average payoff = 34.0436809039
rho = 0.5000,	average payoff = 33.9662958027
rho = 1.0000,	average payoff = 33.9393409320
rho = 2.0000,	average payoff = 33.9625933968
rho = 4.0000,	average payoff = 33.9667788049
rho = 8.0000,	average payoff = 33.9688676579
rho = 16.0000,	average payoff = 33.9656068765
With rho =  0.0625  Average payoff value is =  34.05875409625568


### Kernel CFA - PI = 100 / J = 4 / B = 19 / Optimal Bandwidth

In [176]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 42.9112411624
rho = 0.1250,	average payoff = 42.9045668752
rho = 0.2500,	average payoff = 42.8924331078
rho = 0.5000,	average payoff = 42.8748123567
rho = 1.0000,	average payoff = 42.8664215296
rho = 2.0000,	average payoff = 42.8732726769
rho = 4.0000,	average payoff = 42.8755494904
rho = 8.0000,	average payoff = 42.8806775177
rho = 16.0000,	average payoff = 42.8803677341
With rho =  0.0625  Average payoff value is =  42.9066926313526


### Kernel CFA - PI = 100 / J = 4 / B = 9 / Optimal Bandwidth

In [177]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 42.9116830540
rho = 0.1250,	average payoff = 42.9116830540
rho = 0.2500,	average payoff = 42.9104692035
rho = 0.5000,	average payoff = 42.9025098314
rho = 1.0000,	average payoff = 42.8745997722
rho = 2.0000,	average payoff = 42.8711206390
rho = 4.0000,	average payoff = 42.8797182699
rho = 8.0000,	average payoff = 42.8868817389
rho = 16.0000,	average payoff = 42.8880717179
With rho =  0.0625  Average payoff value is =  42.90830944157098


### Kernel CFA - PI = 100 / J = 4 / B = 3 / Optimal Bandwidth

In [178]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 4, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 42.9116830540
rho = 0.1250,	average payoff = 42.9116830540
rho = 0.2500,	average payoff = 42.9112411624
rho = 0.5000,	average payoff = 42.9012185956
rho = 1.0000,	average payoff = 42.8734151465
rho = 2.0000,	average payoff = 42.8676558420
rho = 4.0000,	average payoff = 42.8766369457
rho = 8.0000,	average payoff = 42.8841741474
rho = 16.0000,	average payoff = 42.8816427348
With rho =  0.0625  Average payoff value is =  42.91054217699352


### Kernel CFA - PI = 110 / J = 4 / B = 19 / Optimal Bandwidth

In [179]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 50.0362582171
rho = 0.1250,	average payoff = 50.0340264399
rho = 0.2500,	average payoff = 50.0324980737
rho = 0.5000,	average payoff = 50.0270108463
rho = 1.0000,	average payoff = 50.0287111148
rho = 2.0000,	average payoff = 50.0296693893
rho = 4.0000,	average payoff = 50.0355847423
rho = 8.0000,	average payoff = 50.0409529919
rho = 16.0000,	average payoff = 50.0440667419
With rho =  16.0  Average payoff value is =  50.04406674192626


### Kernel CFA - PI = 110 / J = 4 / B = 9 / Optimal Bandwidth

In [180]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 50.0362582171
rho = 0.1250,	average payoff = 50.0362582171
rho = 0.2500,	average payoff = 50.0362582171
rho = 0.5000,	average payoff = 50.0362582171
rho = 1.0000,	average payoff = 50.0329347272
rho = 2.0000,	average payoff = 50.0377910945
rho = 4.0000,	average payoff = 50.0442590156
rho = 8.0000,	average payoff = 50.0434438719
rho = 16.0000,	average payoff = 50.0478115082
With rho =  16.0  Average payoff value is =  50.04781150818004


### Kernel CFA - PI = 110 / J = 4 / B = 3 / Optimal Bandwidth

In [181]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 4, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 50.0362582171
rho = 0.1250,	average payoff = 50.0362582171
rho = 0.2500,	average payoff = 50.0362582171
rho = 0.5000,	average payoff = 50.0362582171
rho = 1.0000,	average payoff = 50.0331997526
rho = 2.0000,	average payoff = 50.0333588626
rho = 4.0000,	average payoff = 50.0374204911
rho = 8.0000,	average payoff = 50.0374204911
rho = 16.0000,	average payoff = 50.0392729068
With rho =  16.0  Average payoff value is =  50.039272906777676


### Kernel CFA - PI = 90 / J = 8 / B = 19 / Optimal Bandwidth

In [185]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 45.0403319818
rho = 0.1250,	average payoff = 45.0403319818
rho = 0.2500,	average payoff = 45.0403319818
rho = 0.5000,	average payoff = 45.0338068001
rho = 1.0000,	average payoff = 45.0291442750
rho = 2.0000,	average payoff = 45.0277933771
rho = 4.0000,	average payoff = 45.0297802718
rho = 8.0000,	average payoff = 45.0274150583
rho = 16.0000,	average payoff = 45.0296550179
With rho =  0.0625  Average payoff value is =  45.03602653915658


### Kernel CFA - PI = 90 / J = 8 / B = 9 / Optimal Bandwidth

In [187]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 45.0403319818
rho = 0.1250,	average payoff = 45.0403319818


<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 90 / J = 8 / B = 3 / Optimal Bandwidth

In [188]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 4, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 45.0403319818
rho = 0.1250,	average payoff = 45.0403319818
rho = 0.2500,	average payoff = 45.0403319818
rho = 0.5000,	average payoff = 45.0332740948
rho = 1.0000,	average payoff = 45.0302171227
rho = 2.0000,	average payoff = 45.0302175674
rho = 4.0000,	average payoff = 45.0307247405
rho = 8.0000,	average payoff = 45.0292033515
rho = 16.0000,	average payoff = 45.0292033515
With rho =  0.0625  Average payoff value is =  45.03602653915658


### Kernel CFA - PI = 100 / J = 8 / B = 19 / Optimal Bandwidth

In [189]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 52.4135220082
rho = 0.1250,	average payoff = 52.4135220082


<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 100 / J = 8 / B = 9 / Optimal Bandwidth

In [191]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method
centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
<ipython-input-117-1f1e8d8b8353>:35: RuntimeWarning: invalid value encountered in multiply
  if (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j]) < payoff[i, j]:


rho = 0.0625,	average payoff = 45.0111947583
rho = 0.1250,	average payoff = 45.0403319818


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 100 / J = 8 / B = 3 / Optimal Bandwidth

In [193]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid         = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(rho[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 52.4135220082


<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
<ipython-input-117-1f1e8d8b8353>:35: RuntimeWarning: invalid value encountered in multiply
  if (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j]) < payoff[i, j]:


rho = 0.1250,	average payoff = 52.3779539412


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 110 / J = 8 / B = 19 / Optimal Bandwidth

In [194]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid         = np.percentile(prices, np.arange(1, 20, 1)*5) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(rho[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 57.0781849981


<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 110 / J = 8 / B = 9 / Optimal Bandwidth

In [195]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4]) # bandwidth for kernel method



prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid         = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)
    
    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(rho[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 57.0781849981
rho = 0.1250,	average payoff = 57.0781849981


<ipython-input-115-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)


LinAlgError: SVD did not converge in Linear Least Squares

### Kernel CFA - PI = 110 / J = 8 / B = 3 / Optimal Bandwidth

In [217]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4]) # bandwidth for kernel method


prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid         = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)



average_payoff_list = []
for i in range(len(rho)):
    
    F                   = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices,
    knock_out           = yt,
    payoff              = gt,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    w                   = fit_CFA(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma)

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho[i],
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("rho = {:.4f},\taverage payoff = {:.10f}".format(rho[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma)


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho[max_payoff_ind],
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With rho = ", rho[max_payoff_ind], " Average payoff value is = ", average_payoff)

rho = 0.0625,	average payoff = 57.0781849981


<ipython-input-211-9b22fda3afe4>:26: RuntimeWarning: invalid value encountered in matmul
  ct = gamma*np.max(np.vstack([payoff[:, i + 1], np.matmul(feature_matrix[:, :, i + 1], w[:, i + 1])]), axis = 0)
<ipython-input-213-1f1e8d8b8353>:35: RuntimeWarning: invalid value encountered in multiply
  if (1 - knock_out[i, j])*np.sum(weight[:, j]*feature_matrix[i, :, j]) < payoff[i, j]:


rho = 0.1250,	average payoff = 56.9497903790


LinAlgError: SVD did not converge in Linear Least Squares

# Step 5

### Kernek CFA - PI = 90 / J = 4 / B =  3 / rho =  0.0625 - With Regularization

In [219]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 0.0625 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 34.0784065488
lambda = 0.1250,	average payoff = 34.0784065488
lambda = 0.2500,	average payoff = 34.0784065488
lambda = 0.5000,	average payoff = 34.0784065488
lambda = 2.0000,	average payoff = 34.0784065488
lambda = 4.0000,	average payoff = 34.0784065488
lambda = 8.0000,	average payoff = 34.0784065488
lambda = 16.0000,	average payoff = 34.0784065488
With lambda =  0.0625  Average payoff value is =  34.07840654882202


### Kernek CFA - PI = 100 / J = 4 / B = 3 / rho = 0.0625  - With Regularization

In [220]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 0.0625 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 42.9116830540
lambda = 0.1250,	average payoff = 42.9116830540
lambda = 0.2500,	average payoff = 42.9116830540
lambda = 0.5000,	average payoff = 42.9116830540
lambda = 2.0000,	average payoff = 42.9116830540
lambda = 4.0000,	average payoff = 42.9116830540
lambda = 8.0000,	average payoff = 42.9116830540
lambda = 16.0000,	average payoff = 42.9116830540
With lambda =  0.0625  Average payoff value is =  42.91168305395115


### Kernek CFA - PI = 110 / J = 4 / B = 9 / rho = 16.0 - With Regularization

In [221]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 4    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 16.0 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 10, 1)*10) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 50.0362582171
lambda = 0.1250,	average payoff = 50.0362582171
lambda = 0.2500,	average payoff = 50.0362582171
lambda = 0.5000,	average payoff = 50.0362582171
lambda = 2.0000,	average payoff = 50.0362582171
lambda = 4.0000,	average payoff = 50.0362582171
lambda = 8.0000,	average payoff = 50.0362582171
lambda = 16.0000,	average payoff = 50.0362582171
With lambda =  0.0625  Average payoff value is =  50.036258217082434


### Kernek CFA - PI = 90 / J = 8 / B = 3 / rho = 0.0625 - With Regularization

In [222]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 90.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 0.0625 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 45.0403319818
lambda = 0.1250,	average payoff = 45.0403319818
lambda = 0.2500,	average payoff = 45.0403319818
lambda = 0.5000,	average payoff = 45.0403319818
lambda = 2.0000,	average payoff = 45.0403319818
lambda = 4.0000,	average payoff = 45.0403319818
lambda = 8.0000,	average payoff = 45.0403319818
lambda = 16.0000,	average payoff = 45.0403319818
With lambda =  0.0625  Average payoff value is =  45.04033198184753


### Kernek CFA - PI = 100 / J = 8 / B = 3 / rho = 0.0625 - With Regularization

In [225]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 100.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 0.0625 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 52.4135220082
lambda = 0.1250,	average payoff = 52.4135220082
lambda = 0.2500,	average payoff = 52.4135220082
lambda = 0.5000,	average payoff = 52.4135220082
lambda = 2.0000,	average payoff = 52.4135220082
lambda = 4.0000,	average payoff = 52.4135220082
lambda = 8.0000,	average payoff = 52.4135220082
lambda = 16.0000,	average payoff = 52.4135220082
With lambda =  0.0625  Average payoff value is =  52.41352200820674


### Kernek CFA - PI = 110 / J = 8 / B = 3 / rho = 0.0625 - With Regularization

In [223]:
time_horizon     = 3    # number of years, Y.
num_stages       = 54   # number of exercise opportunities, T.
delta_t          = time_horizon/num_stages   # elapsed time, tau
num_asset        = 8    # number of assets
init_price       = 110.  *  np.ones([num_asset]) # initial price
interest_rate    = 0.05 *  np.ones([num_asset]) # interest rate
sigma            = 0.2  *  np.ones([num_asset]) # volatility
num_trajectories = 3000 # number of trajectories for each asset
barrier_price    = 170. # barrier price for knock-out
strike_price     = 100. # strike price for payoff
gamma            = np.exp(-np.mean(interest_rate)*delta_t) # discount factor
rho              = 0.0625 # bandwidth for kernel method
reg_lambda       = 2.0**np.array([-4, -3, -2, -1, 1, 2, 3, 4])

prices                                  = price_simulator_multiple_trajectories(
                    random_seed         = 123,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

centroid = np.percentile(prices, np.arange(1, 4, 1)*25) # centroid for kernel method

prices_test                             = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)

prices_valid                            = price_simulator_multiple_trajectories(
                    random_seed         = 321,
                    num_asset           = num_asset,
                    init_price          = init_price,
                    interest_rate       = interest_rate,
                    sigma               = sigma,
                    num_stages          = num_stages,
                    delta_t             = delta_t,
                    num_trajectories    = num_trajectories)



yt                                      = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices, 
                    barrier_price       = barrier_price)

gt                                      = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    strike_price        = strike_price)

F                                       = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices,
                    knock_out           = yt,
                    payoff              = gt,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)

average_payoff_list = []
for i in range(len(reg_lambda)):
    w                   = fit_CFA_regularizer(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    feature_matrix      = F,
    payoff              = gt,
    gamma               = gamma,
    reg_lambda          = reg_lambda[i])

    yt_valid            = knock_out_indicator_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid, 
    barrier_price       = barrier_price)

    gt_valid            = payoff_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    strike_price        = strike_price)

    F_valid             = feature_matrix_calculator(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    knock_out           = yt_valid,
    payoff              = gt_valid,
    method              = "kernel",
    rho                 = rho,
    centroid            = centroid)

    average_payoff_list.append(policy_simulation(
    num_asset           = num_asset,
    num_stages          = num_stages,
    num_trajectories    = num_trajectories,
    prices              = prices_valid,
    weight              = w,
    payoff              = gt_valid,
    knock_out           = yt_valid,
    feature_matrix      = F_valid,
    gamma               = gamma))
    print("lambda = {:.4f},\taverage payoff = {:.10f}".format(reg_lambda[i], average_payoff_list[i]))

max_payoff_ind = np.argmax(average_payoff_list)

w                                       = fit_CFA_regularizer(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    feature_matrix      = F,
                    payoff              = gt,
                    gamma               = gamma,
                    reg_lambda          = reg_lambda[max_payoff_ind])


yt_test                                 = knock_out_indicator_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test, 
                    barrier_price       = barrier_price)

gt_test                                 = payoff_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    strike_price        = strike_price)

F_test                                  = feature_matrix_calculator(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    knock_out           = yt_test,
                    payoff              = gt_test,
                    method              = "kernel",
                    rho                 = rho,
                    centroid            = centroid)


average_payoff                          = policy_simulation(
                    num_asset           = num_asset,
                    num_stages          = num_stages,
                    num_trajectories    = num_trajectories,
                    prices              = prices_test,
                    weight              = w,
                    payoff              = gt_test,
                    knock_out           = yt_test,
                    feature_matrix      = F_test,
                    gamma               = gamma)

print("With lambda = ", reg_lambda[max_payoff_ind], " Average payoff value is = ", average_payoff)

lambda = 0.0625,	average payoff = 57.0781849981
lambda = 0.1250,	average payoff = 57.0781849981
lambda = 0.2500,	average payoff = 57.0781849981
lambda = 0.5000,	average payoff = 57.0781849981
lambda = 2.0000,	average payoff = 57.0781849981
lambda = 4.0000,	average payoff = 57.0781849981
lambda = 8.0000,	average payoff = 57.0781849981
lambda = 16.0000,	average payoff = 57.0781849981
With lambda =  0.0625  Average payoff value is =  57.078184998068146
